Run cells bellow to preprocess our **custom** datasets.

Estimated time is around 45 minutes.

In [1]:
import pandas as pd
import os
from pathlib import Path
import gc
gc.collect()

0

All Items

In [2]:
# ds_path = "/home/ubuntu/vasp/Datasets/ml-20m/preprocessed_100"
# ds_path_out = "/home/ubuntu/vasp/Datasets/ml-20m/preprocessed_vasp"

ds_path = "/home/ubuntu/vasp/Datasets/netflix/preprocessed_100"
ds_path_out = "/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp"

# ds_path = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_100"
# ds_path_out = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp"

Path(ds_path_out).mkdir(parents=True, exist_ok=True)

In [3]:
temp_tr = pd.read_csv(os.path.join(ds_path, 'train_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'train_te.csv'))
train = pd.concat([temp_tr, temp_te])
train.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(train.head())
print(len(train))

temp_tr = pd.read_csv(os.path.join(ds_path, 'validation_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'validation_te.csv'))
val = pd.concat([temp_tr, temp_te])
val.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(val.head())
print(len(val))

temp_tr = pd.read_csv(os.path.join(ds_path, 'test_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'test_te.csv'))
test = pd.concat([temp_tr, temp_te])
test.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(test.head())
print(len(test))

full = pd.concat([train, val, test])
print(full.head())
print(len(full))

   userid  itemid                                 title
0  296532     172                   The Devil's Brigade
1  296532     294        Ace Ventura: When Nature Calls
2  296532     328                                 Dogma
3  296532     456                     Kill Bill: Vol. 2
4  296532     526  The Hitchhiker's Guide to the Galaxy
45516928
   userid  itemid                           title
0  409802      29          Something's Gotta Give
1  409802     298           Bridget Jones's Diary
2  409802     312                  Pay It Forward
3  409802     690  The Hand that Rocks the Cradle
4  409802     884                             Ray
5685127
   userid  itemid           title
0  458924    3331     The Village
1  458924    3369     Whale Rider
2  458924    3454  Lost: Season 1
3  458924    3487    Time Bandits
4  458924    3915    Garden State
5677981
   userid  itemid                                 title
0  296532     172                   The Devil's Brigade
1  296532     294      

In [4]:
items = pd.read_csv(os.path.join(ds_path, 'unique_iId.txt'))
items['iId']=items['iId'].apply(str)
items.rename({'iId': 'itemid'}, axis=1, inplace=True)

items.to_json(os.path.join(ds_path_out, 'items.json'))
items

,itemid
0,1
1,2
2,3
3,4
4,5
...,...
17763,17766
17764,17767
17765,17768
17766,17769


In [5]:
result = train['itemid'].isin(test['itemid'])

result.unique()

array([ True, False])

Purchases and grouped purchases.

In [6]:
interactions = full
del full
gc.collect()

interactions['itemid'] = interactions['itemid'].apply(str)
interactions['userid'] = interactions['userid'].apply(str)
interactions.reset_index(drop=True, inplace=True)
interactions.to_json(os.path.join(ds_path_out, 'purchases.json'))
purch = interactions

interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userid','itemids']].drop_duplicates()
iii.reset_index(drop=True, inplace=True)
iii.to_json(os.path.join(ds_path_out, 'purchases_txt.json'))
print(iii.head())

   userid                                            itemids
0  296532  172,294,328,456,526,562,983,1193,1218,1404,161...
1  292402  174,298,312,634,687,987,1041,1312,1643,2126,22...
2   37677  1142,1426,1516,1797,2184,2198,2578,2780,3104,3...
3  188370  190,196,472,569,983,1903,2035,2120,2150,2170,2...
4  338669  759,4430,5315,6035,7603,9324,11520,11817,13072...


Keep only users with 5 or more interactions.

Purchases.

In [10]:
purchases=pd.read_json(os.path.join(ds_path_out, 'purchases.json'))
gc.collect()

purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
#purchases_item_counts = purchases[['uId','iId']]
#purchases_user_counts = purchases[['uId','iId']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_user_count))
pu5=purchases_user_count#[purchases_user_count.nr_of_purchases>=5]
print(len(purchases_user_count))
print(len(pu5))
gc.collect()

purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_item_count_pu5))

#purchases_pu5.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))
purchases.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))

463435
463435
463435
17768


Grouped purchases.

In [11]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userid','itemids']].drop_duplicates()
iii['uId']=iii['userid'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json(os.path.join(ds_path_out, 'purchases_txt_pu5.json'))

Users.

In [12]:
iii['userid'].to_frame().to_json(os.path.join(ds_path_out, 'users_pu5.json'))
print(iii['userid'].to_frame().head())

     userid
0    296532
100  292402
200   37677
268  188370
368  338669


Items

In [13]:
#items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")
items[items.itemid.isin(items.itemid)].to_json(os.path.join(ds_path_out, 'items_pu5.json'))

Items sorted by number of interactions

In [14]:
purchases_item_count_pu5.to_json(os.path.join(ds_path_out, 'items_sorted_pu5.json'))
purchases_item_count_pu5

,itemid,nr_of_purchases
8774,1903,153198
7086,16375,152523
1427,11281,151517
5695,15122,135459
13642,6285,133240
...,...,...
4173,13753,2
11622,4467,2
186,10164,2
1495,11342,2


Users sorted by number of interactions

In [15]:
pu5.to_json(os.path.join(ds_path_out, 'users_sorted_pu5.json'))
pu5

,userid,nr_of_purchases
89990,180989,12206
125748,21317,11423
366370,429730,5818
75221,167697,5094
446152,84443,3989
...,...,...
279345,351407,5
361910,425716,5
109460,198511,5
361896,425703,5


Create train, val and test split.

In [16]:
#users = pd.read_json('users_pu5.json')
#shuffled_users = users.sample(frac=1., random_state=42)
test_users = test['userid'].to_frame().drop_duplicates(subset=['userid'])
val_users = val['userid'].to_frame().drop_duplicates(subset=['userid'])
train_users = train['userid'].to_frame().drop_duplicates(subset=['userid'])

test_users.to_json(os.path.join(ds_path_out, "test_users.json"))
print(test_users.head())
val_users.to_json(os.path.join(ds_path_out, "val_users.json"))
print(val_users.head())
train_users.to_json(os.path.join(ds_path_out, "train_users.json"))
print(train_users.head())

len(train_users),len(val_users),len(test_users)

     userid
0    458924
28   459251
45   448003
145  420321
206  442735
     userid
0    409802
100  401721
144  376290
169  379138
262  406485
     userid
0    296532
100  292402
200   37677
268  188370
368  338669


(370748, 46344, 46343)

List of generated json files with preprocessed data.

In [17]:
!ls /home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/*.json

/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/items.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/items_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/items_sorted_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases_txt.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases_txt_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/test_users.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/train_users.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/users_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/users_sorted_pu5.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/val_users.json
